# Getting Started with PrimeHub Python SDK
PrimeHub Python SDK makes you automation with PrimeHub Platform.

In order to make the SDK working, you have to

- install the library with pip
- create a config file in the ~/.primehub/config.json

# Part 1: prerequisite: Configure the environment.

## 1. Install with pip
Let's install PrimeHub Python SDK with pip.

In [1]:
!pip install primehub-python-sdk

## 2. Request the API Token
In order to get the token, you have to have an account in the PrimeHub cluster, the following process will ask you loing with your account.



In [2]:
# PLEASE UPDATE PRIMEHUB_CLUSTER to your cluster
PRIMEHUB_CLUSTER = 'http://primehub-python-sdk.primehub.io'

In [3]:
import os
from primehub import PrimeHub, PrimeHubConfig

ph = PrimeHub(PrimeHubConfig())
if not os.path.isfile(os.path.join(os.getenv("HOME"), ".primehub/config.json")):
    ph.config.generate(PRIMEHUB_CLUSTER)

In [4]:
ph = PrimeHub(PrimeHubConfig())
if ph.is_ready():
    print("PrimeHub Python SDK setup successfully")
    print("Current Group:", ph.primehub_config.current_group)
else:
    print("PrimeHub Python SDK couldn't get the group information, please check the configuration.")

PrimeHub Python SDK setup successfully
Current Group: {'id': 'ccdd52b8-db16-46ce-8a06-70343c79e82a', 'name': 'phusers', 'displayName': 'primehub users'}


## 3. Check the account is Admin account

Use `ph.me.me` to know that the account is admin account.

In [5]:
account_information = ph.me.me()
account_information['isAdmin']

True

## 4. Clean up the previous testing data.

We need to clean up the previous data to prevent the notebook error,

In [6]:
import pandas as pd

In [7]:
# Clean up "cpu-instance-by-sdk" instancetype data.

df_list = pd.DataFrame.from_records(list(ph.admin.instancetypes.list()))
if len(df_list) > 0:
    target_id = list(df_list.loc[df_list['name'] == "cpu-instance-by-sdk"]["id"])

    if len(target_id) == 1:
        print("clean the instance type data: {}".format(target_id[0]))
        ph.admin.instancetypes.delete(list(target_id)[0])

clean the instance type data: cpu-instance-by-sdk


In [8]:
# Clean up "test_group_from_jupyter" group data.

df_list = pd.DataFrame.from_records(list(ph.admin.groups.list()))
if len(df_list) > 0:
    target_id = list(df_list.loc[df_list['name'] == "test_group_from_jupyter"]["id"])

    if len(target_id) == 1:
        print("clean the group data: {}".format(target_id))
        ph.admin.groups.delete(list(target_id)[0])

clean the group data: ['fa382829-d08a-4421-8de4-82e3231fe7a7']


# Part 2: Instance-type-wise operation
We will test:
    
- Create an instance type with specified CPU/GPU/Memory limits
- Disable the Global and assign the instance type to multiple groups
- Add a new Toleration
- Add a new Node Selector

## 1. Create an instance type with specified CPU/GPU/Memory limits

Use `ph.admin.instancetypes.create` to create the instance type.

In [9]:
# create an instance type
config = {
  "name": "cpu-instance-by-sdk",
  "displayName": "CPU 1",
  "description": "1 vCPU / 1G Memory",
  "cpuLimit": 1,
  "memoryLimit": 1,
  "global": True
}
instancetype = ph.admin.instancetypes.create(config)

In [10]:
instancetype['id']

'cpu-instance-by-sdk'

In [11]:
ph.admin.instancetypes.get(instancetype['id'])

{'id': 'cpu-instance-by-sdk',
 'cpuRequest': None,
 'memoryRequest': None,
 'global': True,
 'groups': [],
 'tolerations': [],
 'nodeSelector': None,
 'name': 'cpu-instance-by-sdk',
 'displayName': 'CPU 1',
 'description': '1 vCPU / 1G Memory',
 'cpuLimit': 1,
 'memoryLimit': 1,
 'gpuLimit': 0}

## 2. Add a new Toleration

Use `ph.admin.instancetypes.update` to add the tolerations information into instancetype.

In [12]:
config = {
  "tolerations": {
    "set": [
      {
        "operator": "Equal",
        "effect": "NoSchedule",
        "key": "nvidia.com/gpu",
        "value": "v100"
      }
    ]
  }
}
ph.admin.instancetypes.update(instancetype['id'], config)

{'id': 'cpu-instance-by-sdk',
 'global': True,
 'cpuRequest': None,
 'memoryRequest': None,
 'groups': [],
 'nodeSelector': {}}

In [13]:
ph.admin.instancetypes.get(instancetype['id'])

{'id': 'cpu-instance-by-sdk',
 'cpuRequest': None,
 'memoryRequest': None,
 'global': True,
 'groups': [],
 'tolerations': [{'key': 'nvidia.com/gpu',
   'value': 'v100',
   'operator': 'Equal',
   'effect': 'NoSchedule'}],
 'nodeSelector': {},
 'name': 'cpu-instance-by-sdk',
 'displayName': 'CPU 1',
 'description': '1 vCPU / 1G Memory',
 'cpuLimit': 1,
 'memoryLimit': 1,
 'gpuLimit': 0}

## 3. Add a new Node Selector

Use `ph.admin.instancetypes.update` to add the Node Selector information into instancetype.

In [14]:
config = {
  "nodeSelector": {"zone": "staging"}
}
ph.admin.instancetypes.update(instancetype['id'], config)

{'id': 'cpu-instance-by-sdk',
 'global': True,
 'cpuRequest': None,
 'memoryRequest': None,
 'groups': [],
 'nodeSelector': {'zone': 'staging'}}

In [15]:
ph.admin.instancetypes.get(instancetype['id'])

{'id': 'cpu-instance-by-sdk',
 'cpuRequest': None,
 'memoryRequest': None,
 'global': True,
 'groups': [],
 'tolerations': [{'key': 'nvidia.com/gpu',
   'value': 'v100',
   'operator': 'Equal',
   'effect': 'NoSchedule'}],
 'nodeSelector': {'zone': 'staging'},
 'name': 'cpu-instance-by-sdk',
 'displayName': 'CPU 1',
 'description': '1 vCPU / 1G Memory',
 'cpuLimit': 1,
 'memoryLimit': 1,
 'gpuLimit': 0}

## 4. Disable the Global and assign the instance type to multiple groups

First, we use `ph.admin.groups.create` to create the group and Use `ph.admin.instancetypes.update` to:
- Disable global instance
- Assign the instance type to multiple groups

In [16]:
# Create a group with admin role
config = {
  "name": "test_group_from_jupyter",
  "displayName": "test_group",
  "enabledDeployment": False,
  "enabledSharedVolume": False,
  "quotaCpu": 0.5,
  "quotaGpu": 0,
  "admins": "",
  "users": {
      "connect": []
  }
}
data = ph.admin.groups.create(config)

In [17]:
import pandas as pd
df_list = pd.DataFrame.from_records(list(ph.admin.groups.list()))
df_list

,id,displayName,name,admins,quotaCpu,quotaGpu,quotaMemory,projectQuotaCpu,projectQuotaGpu,projectQuotaMemory,sharedVolumeCapacity
0,ccdd52b8-db16-46ce-8a06-70343c79e82a,primehub users,phusers,phadmin,NaN,NaN,None,None,None,None,1.0
1,df7fdfe4-5e12-4f1c-aeac-8004160c8e86,test_group,test_group_from_jupyter,,0.5,0.0,None,None,None,None,NaN


In [18]:
instance_connect = []
for list_id in list(df_list['id'])[0:2]:
    instance_connect.append({"id": list_id})
instance_connect

[{'id': 'ccdd52b8-db16-46ce-8a06-70343c79e82a'},
 {'id': 'df7fdfe4-5e12-4f1c-aeac-8004160c8e86'}]

In [19]:
# Update an instance type
config = {
  "global": False,
  "groups": {
    "connect": instance_connect
  },
}
ph.admin.instancetypes.update(instancetype['id'], config)

{'id': 'cpu-instance-by-sdk',
 'global': False,
 'cpuRequest': None,
 'memoryRequest': None,
 'groups': [{'id': 'ccdd52b8-db16-46ce-8a06-70343c79e82a',
   'name': 'phusers',
   'displayName': 'primehub users',
   'quotaCpu': None,
   'quotaGpu': None},
  {'id': 'df7fdfe4-5e12-4f1c-aeac-8004160c8e86',
   'name': 'test_group_from_jupyter',
   'displayName': 'test_group',
   'quotaCpu': 0.5,
   'quotaGpu': 0}],
 'nodeSelector': {'zone': 'staging'}}

In [20]:
ph.admin.instancetypes.get(instancetype['id'])

{'id': 'cpu-instance-by-sdk',
 'cpuRequest': None,
 'memoryRequest': None,
 'global': False,
 'groups': [{'id': 'ccdd52b8-db16-46ce-8a06-70343c79e82a',
   'name': 'phusers',
   'displayName': 'primehub users',
   'quotaCpu': None,
   'quotaGpu': None},
  {'id': 'df7fdfe4-5e12-4f1c-aeac-8004160c8e86',
   'name': 'test_group_from_jupyter',
   'displayName': 'test_group',
   'quotaCpu': 0.5,
   'quotaGpu': 0}],
 'tolerations': [{'key': 'nvidia.com/gpu',
   'value': 'v100',
   'operator': 'Equal',
   'effect': 'NoSchedule'}],
 'nodeSelector': {'zone': 'staging'},
 'name': 'cpu-instance-by-sdk',
 'displayName': 'CPU 1',
 'description': '1 vCPU / 1G Memory',
 'cpuLimit': 1,
 'memoryLimit': 1,
 'gpuLimit': 0}